In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [9]:
df = pd.read_csv("../TheOtherOption/data/all_products_ah_nutr.csv", index_col=0)

/Users/judithvanleersum/.pyenv/versions/lewagon/envs/TheOtherOption/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (42,47,49,50,51,52,53,54,55,57,58,60,61,62,63,64,65,66,67,68,69,70,71,133,143,147,149,151,153,155,157,159,163,165,169,171,173,175,177,179,181,183,185,187,189,191) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
df.head()

,allergy,id,brand,category,description,features,image_url,ingredients,nutrition,summary,...,Nitraten_val,Nitraten_unit,Siliciumdioxode (SIO2)_val,Siliciumdioxode (SIO2)_unit,Sulfaat_val,Sulfaat_unit,Fluoride_val,Fluoride_unit,"Eiwitten, Plantaardig_val","Eiwitten, Plantaardig_unit"
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,no_info,wi54074,AH,"Aardappel, groente, fruit/Komkommer (vers)",AH Komkommer,"['vegetarian', 'free_from_gluten', 'free_from_...",https://static.ah.nl/static/product/AHI_434d50...,no info,no info,"Komkommer heeft een frisse, knapperige, neutra...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,no_info,wi197393,AH,"Aardappel, groente, fruit/Bananen",AH Bananen,"['ah_vegan', 'free_from_milk', 'free_from_glut...",https://static.ah.nl/static/product/AHI_435452...,no info,"{'Eiwitten': '1 g', 'Energie': '385 kJ (91 kca...",Bananen met een zachte en zoete smaak. Lekker ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,no_info,wi4164,AH,"Aardappel, groente, fruit/Courgette (vers)",AH Courgette,"['vegetarian', 'free_from_milk', 'free_from_gl...",https://static.ah.nl/static/product/AHI_434d50...,no info,no info,"De courgette heeft een frisse, zachte, redelij...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,no_info,wi41194,AH,"Aardappel, groente, fruit/Paprika mix (vers)",AH Paprika mix,"['ah_vegan', 'free_from_gluten', 'free_from_mi...",https://static.ah.nl/static/product/AHI_435452...,no info,"{'Eiwitten': '0.8 g', 'Energie': '100 kJ (24 k...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,no_info,wi4177,AH,"Aardappel, groente, fruit/Broccoli (vers)",AH Broccoli,NaN,https://static.ah.nl/static/product/AHI_435452...,no info,no info,"Broccoli heeft een knapperige, frisse iets pit...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
allergies = ['selderij', 'lactose', 'melk. Kan bevatten: eieren', 'vis', 'soja']
allergy_list = []
kan_bevatten_list = []
tag = ""
for a in allergies:
    if tag != "kb":
        if "Kan bevatten" in a:
            tag = "kb"
            new_a = a.split(".")
            for b in new_a:
                if "Kan bevatten" in b:
                    new_b = b.split(":")
                    kan_bevatten_list.append(new_b[1].strip())
                else:
                    allergy_list.append(b.strip())
        else:
            allergy_list.append(a)
    else:
        kan_bevatten_list.append(a)
print(allergy_list)
print(kan_bevatten_list)

['selderij', 'lactose', 'melk']
['eieren', 'vis', 'soja']


In [14]:
df_allergies = df.copy()
all_allergies = []
for ind, allergies in df["allergy"].items():
    allergy_list = []
    kan_bevatten_list = []
    tag = ""
    if allergies == "no_info":
        df_allergies.loc[ind, "allergies"] = "no_info"
        df_allergies.loc[ind, "allergies_kan_bevatten"] = "no_info"
    else:
        for a in eval(allergies):
            if tag != "kb":
                if "Kan bevatten" in a:
                    tag = "kb"
                    new_a = a.split(".")
                    for b in new_a:
                        if "Kan bevatten" in b:
                            new_b = b.split(":")
                            kan_bevatten_list.append(new_b[1].strip())
                            all_allergies.append(new_b[1].strip())
                        else:
                            allergy_list.append(b.strip())
                            all_allergies.append(b.strip())
                else:
                    allergy_list.append(a)
                    all_allergies.append(a)
            else:
                kan_bevatten_list.append(a)
                all_allergies.append(a)
        if kan_bevatten_list == []:
            kan_bevatten_list = "no_info"
        df_allergies.at[ind, "allergies_kan_bevatten"] = kan_bevatten_list
        df_allergies.at[ind, "allergies"] = allergy_list

In [15]:
all_allergies = set(all_allergies)
allergies_df = pd.DataFrame(all_allergies)
allergies_df.to_csv("../TheOtherOption/data/allergies_list.csv", index=False)

In [16]:
for allergy in all_allergies:
    df_allergies[allergy] = ""

In [17]:
for ind, allergy in df_allergies["allergies"].items():
    if allergy != "no_info":
        for a in allergy:
            df_allergies.loc[ind, a] = "allergy"

In [18]:
for ind, allergy in df_allergies["allergies_kan_bevatten"].items():
    if allergy != "no_info":
        for a in allergy:
            df_allergies.loc[ind, a] = "kan bevatten"

In [23]:
df_allergies[["allergy", "sulfieten", "eieren"]]

,allergy,sulfieten,eieren
Unnamed: 0,,,
0,no_info,,
1,no_info,,
2,no_info,,
3,no_info,,
4,no_info,,
...,...,...,...
13905,['sulfieten'],allergy,
13906,['sulfieten'],allergy,
13907,['sulfieten'],allergy,


In [24]:
df_allergies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10883 entries, 0 to 13909
Columns: 224 entries, allergy to sesamzaad
dtypes: float64(60), object(164)
memory usage: 18.9+ MB


In [25]:
df_allergies.to_csv("../TheOtherOption/data/all_products_ah_nutr_allergies.csv", index=False)